In [26]:
import cv2
import numpy as np
import dlib

cap = cv2.VideoCapture(0)
img=cv2.imread('/home/mini/Downloads/glass.png')
gh,gw=img.shape[:2]
imggray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, mask = cv2.threshold(imggray, 200, 255, cv2.THRESH_BINARY_INV)
mask_inv = cv2.bitwise_not(mask)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

while True:
    _, frame = cap.read()
    frame=cv2.flip(frame,1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)
    for face in faces:

        landmarks = predictor(gray, face)
    
        for n in range(0, 68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            glassw=abs(landmarks.part(16).x-landmarks.part(1).x)
            glassh=int(glassw*gh/gw)
            glass=cv2.resize(img,(glassw,glassh))#interpolation=cv2.INTER_AREA)
            mask=cv2.resize(mask,(glassw,glassh))#interpolation=cv2.INTER_AREA)
            mask_inv=cv2.resize(mask_inv,(glassw,glassh))#interpolation=cv2.INTER_AREA)
            
            y1 = int(landmarks.part(24).y)
            y2 = int(y1 + glassh)
            x1 = int(landmarks.part(27).x - (glassw/2))
            x2 = int(x1 + glassw)
            roi1 = frame[y1:y2, x1:x2]
            roi_bg = cv2.bitwise_and(roi1,roi1,mask = mask_inv)
            roi_fg = cv2.bitwise_and(glass,glass,mask = mask)
            frame[y1:y2, x1:x2] = cv2.add(roi_bg, roi_fg)
    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()
cap.release()

In [25]:
cv2.destroyAllWindows()
cap.release()